## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-16-06-51-52 +0000,nypost,Ex-South Korea President Yoon sentenced to 5 y...,https://nypost.com/2026/01/16/world-news/ex-so...
1,2026-01-16-06-49-00 +0000,wsj,U.S. Treasury Yield Curve Is Steepening,https://www.wsj.com/finance/investing/jgbs-fal...
2,2026-01-16-06-43-49 +0000,nypost,"Charles Foehner, NYC senior citizen who fatall...",https://nypost.com/2026/01/16/us-news/charles-...
3,2026-01-16-06-39-53 +0000,bbc,Jenrick defects to Reform UK after being sacke...,https://www.bbc.com/news/articles/cwyrpdy42n1o...
4,2026-01-16-06-21-39 +0000,nypost,Minneapolis fire department report reveals Ren...,https://nypost.com/2026/01/16/us-news/fire-dep...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2387/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
65,trump,58
111,iran,26
46,ice,22
263,new,15
289,venezuela,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
153,2026-01-15-21-46-24 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/15/us-news/venezuel...,143
55,2026-01-16-01-32-41 +0000,bbc,Trump threatens to invoke Insurrection Act to ...,https://www.bbc.com/news/articles/cwy15596p98o...,134
90,2026-01-15-23-31-25 +0000,latimes,Venezuela opposition leader Machado says she '...,https://www.latimes.com/politics/story/2026-01...,130
72,2026-01-16-00-46-28 +0000,wapo,Trump threatens Insurrection Act deployment to...,https://www.washingtonpost.com/nation/2026/01/...,129
304,2026-01-15-13-36-16 +0000,nypost,Trump threatens to invoke Insurrection Act if ...,https://nypost.com/2026/01/15/us-news/trump-th...,126


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
153,143,2026-01-15-21-46-24 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/15/us-news/venezuel...
55,76,2026-01-16-01-32-41 +0000,bbc,Trump threatens to invoke Insurrection Act to ...,https://www.bbc.com/news/articles/cwy15596p98o...
201,71,2026-01-15-19-20-00 +0000,wsj,"In an interview, the Philly Fed’s new presiden...",https://www.wsj.com/economy/central-banking/an...
220,61,2026-01-15-18-04-50 +0000,nypost,White House warns Iran will face ‘grave conseq...,https://nypost.com/2026/01/15/us-news/sen-lind...
4,53,2026-01-16-06-21-39 +0000,nypost,Minneapolis fire department report reveals Ren...,https://nypost.com/2026/01/16/us-news/fire-dep...
185,43,2026-01-15-20-09-34 +0000,bbc,US forces seize a sixth Venezuela-linked oil t...,https://www.bbc.com/news/articles/cy4qdnj5vl9o...
31,40,2026-01-16-03-30-40 +0000,latimes,Trump administration's demands for California'...,https://www.latimes.com/california/story/2026-...
245,37,2026-01-15-17-00-27 +0000,nypost,"Verizon offers ‘pathetic’ $20 credit, sparking...",https://nypost.com/2026/01/15/business/verizon...
276,37,2026-01-15-15-06-43 +0000,nypost,2 dead in Thailand after crane crashes onto ro...,https://nypost.com/2026/01/15/world-news/two-d...
20,35,2026-01-16-04-39-48 +0000,nypost,South Carolina fugitive showed ‘no remorse’ fo...,https://nypost.com/2026/01/15/us-news/south-ca...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
